In [3]:
import csv
import pandas as pd

In [8]:
superstore = pd.read_csv('Superstore.csv', encoding='latin1')
print(superstore.head())

   Row ID        Order ID  Order Date   Ship Date       Ship Mode Customer ID  \
0       1  CA-2013-152156  09-11-2013  12-11-2013    Second Class    CG-12520   
1       2  CA-2013-152156  09-11-2013  12-11-2013    Second Class    CG-12520   
2       3  CA-2013-138688  13-06-2013  17-06-2013    Second Class    DV-13045   
3       4  US-2012-108966  11-10-2012  18-10-2012  Standard Class    SO-20335   
4       5  US-2012-108966  11-10-2012  18-10-2012  Standard Class    SO-20335   

     Customer Name    Segment        Country             City  ...  \
0      Claire Gute   Consumer  United States        Henderson  ...   
1      Claire Gute   Consumer  United States        Henderson  ...   
2  Darrin Van Huff  Corporate  United States      Los Angeles  ...   
3   Sean O'Donnell   Consumer  United States  Fort Lauderdale  ...   
4   Sean O'Donnell   Consumer  United States  Fort Lauderdale  ...   

  Postal Code  Region       Product ID         Category Sub-Category  \
0       42420   Sout

In [14]:
superstore.columns

Index(['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State',
       'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category',
       'Product Name', 'Sales', 'Quantity', 'Discount', 'Profit'],
      dtype='str')

### 1. Structure

In [9]:
# 1.1 How many rows and columns are in the dataset?
print(superstore.shape)

(9994, 21)


In [ ]:
# 1.2 Is it transaction-level (one row per order line)?
superstore['Order ID'].nunique(), superstore.shape[0]
# This shows that there are multiple rows per order, confirming that it is transaction-level data.

(5009, 9994)

### 2. Time

In [ ]:
#2.1 What's the date range?
superstore['Order Date'] = pd.to_datetime(superstore['Order Date'], format='%d-%m-%Y')
superstore['Order Date'].min(), superstore['Order Date'].max()
# The dataset spans from January 1, 2011 to December 31, 2014.

(Timestamp('2011-01-04 00:00:00'), Timestamp('2014-12-31 00:00:00'))

In [ ]:
#2.2 Is there one date column or multiple?
# There are two date columns: 'Order Date' and 'Ship Date'.

### 3. Finance Fields

##### Financial Fields Overview
The dataset contains several key financial fields at the transaction (line-item) level:
- **Sales**: Revenue generated per product line item
- **Profit**: Net profit per line item after discounts
- **Quantity**: Number of units sold
- **Discount**: Discount applied to the transaction

In [24]:
superstore[['Sales', 'Profit', 'Quantity', 'Discount']].describe()


,Sales,Profit,Quantity,Discount
count,9994.000000,9994.000000,9994.000000,9994.000000
mean,229.858001,28.656896,3.789574,0.156203
std,623.245101,234.260108,2.225110,0.206452
min,0.444000,-6599.978000,1.000000,0.000000
25%,17.280000,1.728750,2.000000,0.000000
50%,54.490000,8.666500,3.000000,0.200000
75%,209.940000,29.364000,5.000000,0.200000
max,22638.480000,8399.976000,14.000000,0.800000


Sales and profit are available at the line-item level, enabling analysis of revenue and profitability drivers across products, regions, and time. Profit includes both positive and negative values, allowing identification of loss-making transactions.

However, the dataset does not include explicit cost or price-per-unit fields. As a result, margin analysis is limited to profit-based metrics rather than detailed cost structure analysis.

Moreover, sales and profit distributions appear right-skewed, with a small number of high-value transactions driving a large share of total revenue and profit.

### 4. Dimensions
: What dimensions are available to slice financial performance, and are they usable?

##### Core Business Dimensions
1. **Product**: `Category`, `Sub-Category`, `Product Name`
2. **Geography**: `Region`, `State`, `City`
3. **Customer**: `Segment`, `CustomerID`

In [ ]:
superstore['Category'].value_counts()

Category
Office Supplies    6026
Furniture          2121
Technology         1847
Name: count, dtype: int64

In [35]:
superstore['Sub-Category'].value_counts()

Sub-Category
Binders        1523
Paper          1370
Furnishings     957
Phones          889
Storage         846
Art             796
Accessories     775
Chairs          617
Appliances      466
Labels          364
Tables          319
Envelopes       254
Bookcases       228
Fasteners       217
Supplies        190
Machines        115
Copiers          68
Name: count, dtype: int64

In [32]:
print(superstore['Region'].value_counts())
print("___________________________")
print(superstore['State'].value_counts())

Region
West       3203
East       2848
Central    2323
South      1620
Name: count, dtype: int64
___________________________
State
California              2001
New York                1128
Texas                    985
Pennsylvania             587
Washington               506
Illinois                 492
Ohio                     469
Florida                  383
Michigan                 255
North Carolina           249
Arizona                  224
Virginia                 224
Georgia                  184
Tennessee                183
Colorado                 182
Indiana                  149
Kentucky                 139
Massachusetts            135
New Jersey               130
Oregon                   124
Wisconsin                110
Maryland                 105
Delaware                  96
Minnesota                 89
Connecticut               82
Missouri                  66
Oklahoma                  66
Alabama                   61
Arkansas                  60
Rhode Island              56

In [36]:
print(superstore['Segment'].value_counts())
print("___________________________")
print(superstore['Customer ID'].nunique())

Segment
Consumer       5191
Corporate      3020
Home Office    1783
Name: count, dtype: int64
___________________________
793


### 5. Initial Inspection Summary

The dataset is transaction-level retail sales data spanning multiple years, with sufficient temporal coverage to analyze revenue and profitability trends over time. Core financial fields (sales and profit) are available at the line-item level, enabling identification of revenue drivers and loss-making transactions, though detailed cost structure analysis is not possible due to missing cost fields.

The dataset includes well-defined product, geographic, and customer dimensions, allowing financial performance to be analyzed across categories, regions, and customer segments. Based on this inspection, category- and region-level analyses are likely to yield the most interpretable insights, while product-level analysis should be aggregated to avoid excessive granularity.
